1. import các thư viện cần thiết

In [1]:
import pytesseract
from PIL import Image
import argparse
import os
import platform
import sys
from pathlib import Path
import numpy as np
import torch
import torch.backends.cudnn as cudnn
import matplotlib.pyplot as plt
from torch import cudnn_affine_grid_generator
import cv2
from models.common import DetectMultiBackend
from utils.dataloaders import IMG_FORMATS, VID_FORMATS, LoadImages, LoadStreams
from utils.general import (LOGGER, check_file, check_img_size, check_imshow, check_requirements, colorstr, 
                           increment_path, non_max_suppression, print_args, strip_optimizer, xyxy2xywh)
from utils.plots import Annotator, colors, save_one_box
from utils.torch_utils import select_device, smart_inference_mode, time_sync

def scale_coords(img1_shape, coords, img0_shape, ratio_pad=None):
    # Rescale coords (xyxy) from img1_shape to img0_shape
    if ratio_pad is None:  # calculate from img0_shape
        gain = min(img1_shape[0] / img0_shape[0], img1_shape[1] / img0_shape[1])  # gain  = old / new
        pad = (img1_shape[1] - img0_shape[1] * gain) / 2, (img1_shape[0] - img0_shape[0] * gain) / 2  # wh padding
    else:
        gain = ratio_pad[0][0]
        pad = ratio_pad[1]

    coords[:, [0, 2]] -= pad[0]  # x padding
    coords[:, [1, 3]] -= pad[1]  # y padding
    coords[:, :4] /= gain
    clip_coords(coords, img0_shape)
    return coords

def clip_coords(boxes, shape):
    # Clip bounding xyxy bounding boxes to image shape (height, width)
    if isinstance(boxes, torch.Tensor):  # faster individually
        boxes[:, 0].clamp_(0, shape[1])  # x1
        boxes[:, 1].clamp_(0, shape[0])  # y1
        boxes[:, 2].clamp_(0, shape[1])  # x2
        boxes[:, 3].clamp_(0, shape[0])  # y2
    else:  # np.array (faster grouped)
        boxes[:, [0, 2]] = boxes[:, [0, 2]].clip(0, shape[1])  # x1, x2
        boxes[:, [1, 3]] = boxes[:, [1, 3]].clip(0, shape[0])  # y1, y2

2. dùng model yolov5 để phân vùng ảnh

In [2]:
@smart_inference_mode()
def run(
        weights='E:/yolov5/runs/train/exp2/weights/best.pt',  # model.pt path(s)
        source='data/images/test.jpg',  # file/dir/URL/glob, 0 for webcam
        data='data/custom_data.yaml',  # dataset.yaml path
        imgsz=(640, 640),  # inference size (height, width)
        conf_thres=0.33,  # confidence threshold
        iou_thres=0.45,  # NMS IOU threshold
        max_det=1000,  # maximum detections per image
        device='',  # cuda device, i.e. 0 or 0,1,2,3 or cpu
        view_img=False,  # show results
        save_txt=False,  # save results to *.txt
        save_conf=False,  # save confidences in --save-txt labels
        save_crop=True,  # save cropped prediction boxes
        nosave=False,  # do not save images/videos
        classes=None,  # filter by class: --class 0, or --class 0 2 3
        agnostic_nms=False,  # class-agnostic NMS
        augment=False,  # augmented inference
        visualize=False,  # visualize features
        update=False,  # update all models
        project='/runs/detect/exp2',  # save results to project/name
        #name='exp',  # save results to project/name
        exist_ok=False,  # existing project/name ok, do not increment
        line_thickness=3,  # bounding box thickness (pixels)
        hide_labels=False,  # hide labels
        hide_conf=False,  # hide confidences
        half=False,  # use FP16 half-precision inference
        dnn=False,  # use OpenCV DNN for ONNX inference
):
    source = str(source)
    save_img = not nosave and not source.endswith('.txt')  # save inference images
    is_file = Path(source).suffix[1:] in (IMG_FORMATS + VID_FORMATS)
    is_url = source.lower().startswith(('rtsp://', 'rtmp://', 'http://', 'https://'))
    webcam = source.isnumeric() or source.endswith('.txt') or (is_url and not is_file)
    if is_url and is_file:
        source = check_file(source)  # download

    # Directories
    save_dir = increment_path(Path(project), exist_ok=exist_ok)  # increment run
    (save_dir / 'labels' if save_txt else save_dir).mkdir(parents=True, exist_ok=True)  # make dir

    # Load model
    device = select_device(device)
    model = DetectMultiBackend(weights, device=device, dnn=dnn, data=data, fp16=half)
    stride, names, pt = model.stride, model.names, model.pt
    imgsz = check_img_size(imgsz, s=stride)  # check image size

    # Dataloader
    if webcam:
        view_img = check_imshow()
        cudnn_affine_grid_generator.benchmark = True  # set True to speed up constant image size inference
        dataset = LoadStreams(source, img_size=imgsz, stride=stride, auto=pt)
        bs = len(dataset)  # batch_size
    else:
        dataset = LoadImages(source, img_size=imgsz, stride=stride, auto=pt)
        bs = 1  # batch_size
    vid_path, vid_writer = [None] * bs, [None] * bs

    # Run inference
    model.warmup(imgsz=(1 if pt else bs, 3, *imgsz))  # warmup
    seen, windows, dt = 0, [], [0.0, 0.0, 0.0]
    for path, im, im0s, vid_cap, s in dataset:
        t1 = time_sync()
        im = torch.from_numpy(im).to(device)
        im = im.half() if model.fp16 else im.float()  # uint8 to fp16/32
        im /= 255  # 0 - 255 to 0.0 - 1.0
        if len(im.shape) == 3:
            im = im[None]  # expand for batch dim
        t2 = time_sync()
        dt[0] += t2 - t1

        # Inference
        visualize = increment_path(save_dir / Path(path).stem, mkdir=True) if visualize else False
        pred = model(im, augment=augment, visualize=visualize)
        t3 = time_sync()
        dt[1] += t3 - t2

        # NMS
        pred = non_max_suppression(pred, conf_thres, iou_thres, classes, agnostic_nms, max_det=max_det)
        dt[2] += time_sync() - t3

        # Second-stage classifier (optional)
        # pred = utils.general.apply_classifier(pred, classifier_model, im, im0s)

        # Process predictions
        for i, det in enumerate(pred):
            print(i)
            print('=======\n')
            print(det.cpu().detach().numpy())
            seen += 1
            if webcam:  # batch_size >= 1
                p, im0, frame = path[i], im0s[i].copy(), dataset.count
                s += f'{i}: '
            else:
                p, im0, frame = path, im0s.copy(), getattr(dataset, 'frame', 0)

            p = Path(p)  # to Path
            save_path = str(save_dir / p.name)  # im.jpg
            txt_path = str(save_dir / 'labels' / p.stem) + ('' if dataset.mode == 'image' else f'_{frame}')  # im.txt
            s += '%gx%g ' % im.shape[2:]  # print string
            gn = torch.tensor(im0.shape)[[1, 0, 1, 0]]  # normalization gain whwh
            imc = im0.copy() if save_crop else im0  # for save_crop
            annotator = Annotator(im0, line_width=line_thickness, example=str(names))
            print('save crop: ',save_crop)
            if len(det):
                # Rescale boxes from img_size to im0 size
                det[:, :4] = scale_coords(im.shape[2:], det[:, :4], im0.shape).round()

                # Print results
                for c in det[:, -1].unique():
                    n = (det[:, -1] == c).sum()  # detections per class
                    s += f"{n} {names[int(c)]}{'s' * (n > 1)}, "  # add to string

                # Write results
                for *xyxy, conf, cls in reversed(det):
                    if save_txt:  # Write to file
                        xywh = (xyxy2xywh(torch.tensor(xyxy).view(1, 4)) / gn).view(-1).tolist()  # normalized xywh
                        line = (cls, *xywh, conf) if save_conf else (cls, *xywh)  # label format
                        with open(f'{txt_path}.txt', 'a') as f:
                            f.write(('%g ' * len(line)).rstrip() % line + '\n')

                    if save_img or save_crop or view_img:  # Add bbox to image
                        c = int(cls)  # integer class
                        label = None if hide_labels else (names[c] if hide_conf else f'{names[c]} {conf:.2f}')
                        annotator.box_label(xyxy, label, color=colors(c, True))
                    if save_crop:
                        save_one_box(xyxy, imc, file=save_dir / 'crops' / names[c] / f'{p.stem}.jpg', BGR=True)

            # Stream results
            im0 = annotator.result()
            if view_img:
                if platform.system() == 'Linux' and p not in windows:
                    windows.append(p)
                    cv2.namedWindow(str(p), cv2.WINDOW_NORMAL | cv2.WINDOW_KEEPRATIO)  # allow window resize (Linux)
                    cv2.resizeWindow(str(p), im0.shape[1], im0.shape[0])
                cv2.imshow(str(p), im0)
                cv2.waitKey(1)  # 1 millisecond

            # Save results (image with detections)
            if save_img:
                if dataset.mode == 'image':
                    cv2.imwrite(save_path, im0)
                else:  # 'video' or 'stream'
                    if vid_path[i] != save_path:  # new video
                        vid_path[i] = save_path
                        if isinstance(vid_writer[i], cv2.VideoWriter):
                            vid_writer[i].release()  # release previous video writer
                        if vid_cap:  # video
                            fps = vid_cap.get(cv2.CAP_PROP_FPS)
                            w = int(vid_cap.get(cv2.CAP_PROP_FRAME_WIDTH))
                            h = int(vid_cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
                        else:  # stream
                            fps, w, h = 30, im0.shape[1], im0.shape[0]
                        save_path = str(Path(save_path).with_suffix('.mp4'))  # force *.mp4 suffix on results videos
                        vid_writer[i] = cv2.VideoWriter(save_path, cv2.VideoWriter_fourcc(*'mp4v'), fps, (w, h))
                    vid_writer[i].write(im0)

        # Print time (inference-only)
        LOGGER.info(f'{s}Done. ({t3 - t2:.3f}s)')

    # Print results
    t = tuple(x / seen * 1E3 for x in dt)  # speeds per image
    LOGGER.info(f'Speed: %.1fms pre-process, %.1fms inference, %.1fms NMS per image at shape {(1, 3, *imgsz)}' % t)
    if save_txt or save_img:
        s = f"\n{len(list(save_dir.glob('labels/*.txt')))} labels saved to {save_dir / 'labels'}" if save_txt else ''
        LOGGER.info(f"Results saved to {colorstr('bold', save_dir)}{s}")
    if update:
        strip_optimizer(weights[0])  # update model (to fix SourceChangeWarning)
run(
        weights='E:/yolov5/runs/train/exp2/weights/best.pt',  # model.pt path(s)
        source='data/images/test.jpg',  # file/dir/URL/glob, 0 for webcam
        data='data/custom_data.yaml',  # dataset.yaml path
        imgsz=(640, 640),  # inference size (height, width)
        conf_thres=0.33,  # confidence threshold
        iou_thres=0.45,  # NMS IOU threshold
        max_det=1000,  # maximum detections per image
        device='',  # cuda device, i.e. 0 or 0,1,2,3 or cpu
        view_img=False,  # show results
        save_txt=False,  # save results to *.txt
        save_conf=False,  # save confidences in --save-txt labels
        save_crop=True,  # save cropped prediction boxes
        nosave=False,  # do not save images/videos
        classes=None,  # filter by class: --class 0, or --class 0 2 3
        agnostic_nms=False,  # class-agnostic NMS
        augment=False,  # augmented inference
        visualize=False,  # visualize features
        update=False,  # update all models
        project='/runs/detect/exp2',  # save results to project/name
        #name='exp',  # save results to project/name
        exist_ok=False,  # existing project/name ok, do not increment
        line_thickness=3,  # bounding box thickness (pixels)
        hide_labels=False,  # hide labels
        hide_conf=False,  # hide confidences
        half=False,  # use FP16 half-precision inference
        dnn=False,  # use OpenCV DNN for ONNX inference
)

YOLOv5  2023-5-22 Python-3.10.6 torch-2.0.1+cpu CPU

Fusing layers... 
Model summary: 157 layers, 7029004 parameters, 0 gradients
image 1/1 E:\yolov5\data\images\test.jpg: 640x320 1 STK ch, 1 ni dung chuyn khon, 1 s tin, 1 thi gian chuyn khon, 1 tn ngi nhn, Done. (0.159s)
Speed: 4.0ms pre-process, 158.6ms inference, 11.0ms NMS per image at shape (1, 3, 640, 640)
Results saved to \runs\detect\exp2


0

[[     190.12      424.46      301.77      450.17     0.95444           2]
 [      107.7      187.79      212.35      206.27     0.87356           3]
 [     213.78      293.07      303.58      306.58     0.86338           1]
 [     168.99      254.42      298.86      268.38     0.82526           6]
 [     105.16      209.91      214.82      220.84     0.68066           4]]
save crop:  True


In [3]:
folder_name1 = 'E:/runs/detect/exp2/crops/tên người chuyển khoản'
folder_name2 = 'E:/runs/detect/exp2/crops/tên người nhận'
folder_name3 = 'E:/runs/detect/exp2/crops/STK nguồn'
folder_name4 = 'E:/runs/detect/exp2/crops/STK đích'
folder_name5 = 'E:/runs/detect/exp2/crops/nội dung chuyển khoản'
folder_name6 = 'E:/runs/detect/exp2/crops/thời gian chuyển khoản'
folder_name7 = 'E:/runs/detect/exp2/crops/số tiền'
# Kiểm tra xem thư mục đã tồn tại hay chưa
if not os.path.exists(folder_name1):
    os.mkdir(folder_name1)
if not os.path.exists(folder_name2):
    os.mkdir(folder_name2)
if not os.path.exists(folder_name3):
    os.mkdir(folder_name3)
if not os.path.exists(folder_name4):
    os.mkdir(folder_name4)
if not os.path.exists(folder_name5):
    os.mkdir(folder_name5)    
if not os.path.exists(folder_name6):
    os.mkdir(folder_name6)
if not os.path.exists(folder_name7):
    os.mkdir(folder_name7)

In [4]:
import os
old_folder_name1 = 'E:/runs/detect/exp2/crops/tên người nhận'
new_folder_name2 = 'E:/runs/detect/exp2/crops/ten-nguoi-nhan'
# Đổi tên thư mục
os.rename(old_folder_name1, new_folder_name2)

old_folder_name1 = 'E:/runs/detect/exp2/crops/tên người chuyển khoản'
new_folder_name2 = 'E:/runs/detect/exp2/crops/ten-nguoi-chuyen-khoan'
# Đổi tên thư mục
os.rename(old_folder_name1, new_folder_name2)

old_folder_name1 = 'E:/runs/detect/exp2/crops/STK nguồn'
new_folder_name2 = 'E:/runs/detect/exp2/crops/STK-nguon'
# Đổi tên thư mục
os.rename(old_folder_name1, new_folder_name2)

old_folder_name1 = 'E:/runs/detect/exp2/crops/STK đích'
new_folder_name2 = 'E:/runs/detect/exp2/crops/STK-dich'
# Đổi tên thư mục
os.rename(old_folder_name1, new_folder_name2)

old_folder_name1 = 'E:/runs/detect/exp2/crops/số tiền'
new_folder_name2 = 'E:/runs/detect/exp2/crops/so-tien'
# Đổi tên thư mục
os.rename(old_folder_name1, new_folder_name2)

old_folder_name1 = 'E:/runs/detect/exp2/crops/nội dung chuyển khoản'
new_folder_name2 = 'E:/runs/detect/exp2/crops/noi-dung-chuyen-khoan'
# Đổi tên thư mục
os.rename(old_folder_name1, new_folder_name2)

old_folder_name1 = 'E:/runs/detect/exp2/crops/thời gian chuyển khoản'
new_folder_name2 = 'E:/runs/detect/exp2/crops/thoi-gian-chuyen-khoan'
# Đổi tên thư mục
os.rename(old_folder_name1, new_folder_name2)



In [7]:
import json

if os.path.exists('E:/runs/detect/exp2/crops/noi-dung-chuyen-khoan/test.jpg'):
    image = cv2.imread('E:/runs/detect/exp2/crops/noi-dung-chuyen-khoan/test.jpg')
    gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    # Convert image to text
    text1 = pytesseract.image_to_string(image, lang='vie', config='--psm 6')     
else:
    text1 = 'null'
    
if os.path.exists('E:/runs/detect/exp2/crops/so-tien/test.jpg'):
    image = cv2.imread('E:/runs/detect/exp2/crops/so-tien/test.jpg')
    gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    # Convert image to text
    text2 = pytesseract.image_to_string(image, lang='vie', config='--psm 6')
else:
    text2 = 'null'   

if os.path.exists('E:/runs/detect/exp2/crops/STK-dich/test.jpg'):
    image = cv2.imread('E:/runs/detect/exp2/crops/STK-dich/test.jpg')
    gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    # Convert image to text
    text3 = pytesseract.image_to_string(image, lang='vie', config='--psm 6')
else:
    text3 = 'null'   

if os.path.exists('E:/runs/detect/exp2/crops/STK-nguon/test.jpg'):
    image = cv2.imread('E:/runs/detect/exp2/crops/STK-nguon/test.jpg')
    gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    # Convert image to text
    text4 = pytesseract.image_to_string(image, lang='vie', config='--psm 6')
else:
    text4 = 'null'    

if os.path.exists('E:/runs/detect/exp2/crops/ten-nguoi-chuyen-khoan/test.jpg'):
    image = cv2.imread('E:/runs/detect/exp2/crops/ten-nguoi-chuyen-khoan/test.jpg')
    gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    # Convert image to text
    text5 = pytesseract.image_to_string(image, lang='vie', config='--psm 6')
else:
    text5 = 'null'   

if os.path.exists('E:/runs/detect/exp2/crops/ten-nguoi-nhan/test.jpg'):
    image = cv2.imread('E:/runs/detect/exp2/crops/ten-nguoi-nhan/test.jpg')
    gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    # Convert image to text
    text6 = pytesseract.image_to_string(image, lang='vie', config='--psm 6')
else:
    text6 = 'null'    

if os.path.exists('E:/runs/detect/exp2/crops/thoi-gian-chuyen-khoan/test.jpg'):
    image = cv2.imread('E:/runs/detect/exp2/crops/thoi-gian-chuyen-khoan/test.jpg')
    gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    # Convert image to text
    text7 = pytesseract.image_to_string(image, lang='vie', config='--psm 6')
else:
    text7 = 'null'   

data = {"nội dung chuyển khoản": text1.strip(),
        "số tiền": text2.strip(),
        "STK đích": text3.strip(),
        "STK nguồn": text4.strip(),
        "tên người chuyển khoản": text5.strip(),
        "tên người nhận": text6.strip(),
        "thời gian chuyển khoản": text7.strip()}
json_data = json.dumps(data, ensure_ascii=False)
print(json_data)

{"nội dung chuyển khoản": "NGUYEN HOAI NAM\nchuyen tien", "số tiền": "", "STK đích": "03269426688", "STK nguồn": "null", "tên người chuyển khoản": "null", "tên người nhận": "DOAN THỊ HONG NGOC", "thời gian chuyển khoản": "18:40 Thứ Sáu 05/05/2023"}
